In [2]:
#!pip install findspark
!pip install numpy

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [3]:
if __name__ == '__main__':
    spark = SparkSession.\
            builder.\
            appName('PySparkLinReg').\
            getOrCreate()

In [4]:
dataset = spark.read.csv(r"C:\Users\siddhant.pathak\Ineuron_PySpark\pyspark_examples\Admission_Prediction.csv", header=True)
dataset

DataFrame[GRE Score: string, TOEFL Score: string, University Rating: string, SOP: string, LOR: string, CGPA: string, Research: string, Chance of Admit: string]

In [5]:
dataset.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [6]:
dataset.printSchema()
dataset.columns

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [7]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast('float').alias(c) for c in dataset.columns))
new_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|
|     null|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|             null|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|              2.0|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|              1.0|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|              3.0

In [8]:
# Counting Missing Values
from pyspark.sql.functions import col, count, isnan, when
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [9]:
# Handling missing values
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['GRE Score', 'TOEFL Score', 'University Rating'], 
                  outputCols=['GRE Score', 'TOEFL Score', 'University Rating'])

imputed_data = imputer.fit(new_data).transform(new_data)
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [10]:
# Dropping column
features = imputed_data.drop('Chance of Admit')
features

DataFrame[GRE Score: float, TOEFL Score: float, University Rating: float, SOP: float, LOR: float, CGPA: float, Research: float]

In [11]:
# Vector Assebler
assembler = VectorAssembler(inputCols=features.columns, outputCol='features')
output = assembler.transform(imputed_data)
output = output.select('features', 'Chance of Admit')
output.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[337.0,118.0,4.0,...|           0.92|
|[324.0,107.0,4.0,...|           0.76|
|[316.558776855468...|           0.72|
|[322.0,110.0,3.0,...|            0.8|
|[314.0,103.0,2.0,...|           0.65|
|[330.0,115.0,5.0,...|            0.9|
|[321.0,109.0,3.12...|           0.75|
|[308.0,101.0,2.0,...|           0.68|
|[302.0,102.0,1.0,...|            0.5|
|[323.0,108.0,3.0,...|           0.45|
|[325.0,106.0,3.0,...|           0.52|
|[327.0,111.0,4.0,...|           0.84|
|[316.558776855468...|           0.78|
|[307.0,109.0,3.0,...|           0.62|
|[311.0,104.0,3.0,...|           0.61|
|[314.0,105.0,3.0,...|           0.54|
|[317.0,107.0,3.0,...|           0.66|
|[319.0,106.0,3.0,...|           0.65|
|[318.0,110.0,3.0,...|           0.63|
|[303.0,102.0,3.0,...|           0.62|
+--------------------+---------------+
only showing top 20 rows



In [12]:
# Splitting Dataset
train_df, test_df = output.randomSplit([0.7, 0.3])

In [13]:
# Linear Regression
lin_reg = LinearRegression(featuresCol='features', labelCol='Chance of Admit', predictionCol='prediction')
lin_model = lin_reg.fit(train_df)
lin_model

IllegalArgumentException: 'Unsupported class file major version 58'